In [1]:
import numpy as np
import math
from typing import List

In [2]:
# Sets up maze grid
X = 1
_ = 0

"""
TODO: You can change up the grid maze to test different expansions.
"""
GRID = np.array([
  [_,X,X,_,_,_,_,_,_,_,X,X,_,_,_,_,],
  [_,X,X,_,_,_,_,_,_,X,X,_,_,_,_,_,],
  [_,X,X,_,_,_,_,_,X,X,_,_,_,_,_,_,],
  [_,X,X,_,_,_,_,X,X,_,_,_,X,X,X,_,],
  [_,X,X,_,_,_,X,X,_,_,_,X,X,X,_,_,],
  [_,X,X,_,_,X,X,_,_,_,X,X,X,_,_,_,],
  [_,X,X,_,X,X,_,_,_,X,X,X,_,_,_,_,],
  [_,X,X,X,X,_,_,_,X,X,X,_,_,_,_,_,],
  [_,X,X,X,_,_,_,X,X,X,_,_,_,_,_,_,],
  [_,X,X,_,_,_,X,X,X,_,_,X,X,X,X,X,],
  [_,X,_,_,_,X,X,X,_,_,X,X,X,X,X,X,],
  [_,_,_,_,X,X,X,_,_,X,X,X,X,X,X,X,],
  [_,_,_,X,X,X,_,_,X,X,X,X,X,X,X,X,],
  [_,_,X,X,X,_,_,X,X,X,X,X,X,X,X,X,],
  [_,X,X,X,_,_,_,_,_,_,_,_,_,_,_,_,],
  [X,X,X,_,_,_,_,_,_,_,_,_,_,_,_,_,]])

START = [0.0, 0.0, 0.0]
GOAL = [len(GRID) - 1, len(GRID[0]) - 1]

In [3]:
from numpy.core.fromnumeric import nonzero
# HBF structs
class maze_s:
  def __init__(self, g:int=0, x:float=0.0, y:float=0.0, theta:float=0.0):
    self.g = g
    self.x = x
    self.y = y
    self.theta = theta
  
class maze_path:
  def __init__(self, closed:List[List[List[int]]]=None, came_from:List[List[List[maze_s]]]=None, final:maze_s=None):
    #if closed is None:         
    #if came_from is None:
    if final is None:
      self.final = maze_s()           
    self.closed = closed
    self.came_from = came_from
    self.final = final

class HBF:
  def __init__(self):
    self.NUM_THETA_CELLS = 90
    self.SPEED = 1.45
    self.LENGTH = 0.5

  def theta_to_stack_number(self, theta: float) -> int:
    # Takes an angle (in radians) and returns which "stack" in the 3D 
    # configuration space this angle corresponds to. Angles near 0 go in the 
    # lower stacks while angles near 2 * pi go in the higher stacks.
    new_theta = (theta + 2 * math.pi) % (2 * math.pi)
    stack_number = int(round(new_theta * self.NUM_THETA_CELLS / (2 * math.pi))) % self.NUM_THETA_CELLS
    return stack_number

  def idx(self, float_num: float) -> int:
    # Returns the index into the grid for continuous position. So if x is 3.621, 
    # then this would return 3 to indicate that 3.621 corresponds to array 
    # index 3.
    return int(math.floor(float_num))

  def expand(self, state) -> List[maze_s]:
    g = state.g
    x = state.x
    y = state.y
    theta = state.theta

    g2 = g + 1
    next_states = []
    for delta_i in range(-35, 40, 5):
      delta = math.pi / 180.0 * delta_i
      omega = self.SPEED / self.LENGTH * math.tan(delta)
      theta2 = theta + omega
      if theta2 < 0:
        theta2 += 2 * math.pi
      x2 = x + self.SPEED * math.cos(theta)
      y2 = y + self.SPEED * math.sin(theta)
      state2 = maze_s(g2, x2, y2, theta2) 
      next_states.append(state2)
    return next_states

  def reconstruct_path(self, came_from: List[List[List[maze_s]]], start: List[float], final: maze_s) -> List[maze_s]:
      path = [final]
      stack = self.theta_to_stack_number(final.theta)
      current = came_from[stack][self.idx(final.x)][self.idx(final.y)]
      stack = self.theta_to_stack_number(current.theta)

      x = current.x
      y = current.y

      while x != start[0] or y != start[1]:
        path.append(current)
        current = came_from[stack][self.idx(x)][self.idx(y)]
        x = current.x
        y = current.y
        stack = self.theta_to_stack_number(current.theta)
      return path

  def search(self, grid: List[List[int]], start: List[float], goal: List[int]):
    # Working Implementation of breadth first search. Does NOT use a heuristic
    # and as a result this is pretty inefficient. Try modifying this algorithm
    # into hybrid A* by adding heuristics appropriately.

    # TODO: Add heuristics and convert this function into hybrid A*
    closed = [[[0 for i in range(len(grid[0]))] for j in range(len(grid))] for k in range(self.NUM_THETA_CELLS)]
    came_from = [[[maze_s() for i in range(len(grid[0]))] for j in range(len(grid))] for k in range(self.NUM_THETA_CELLS)]
    theta = start[2]
    stack = self.theta_to_stack_number(theta)
    g = 0

    state = maze_s()
    state.g = g
    state.x = start[0]
    state.y = start[1]
    state.theta = theta
    
    closed[stack][self.idx(state.x)][self.idx(state.y)] = 1
    came_from[stack][self.idx(state.x)][self.idx(state.y)] = state
    total_closed = 1
    opened = [state]
    finished = False
    while opened:
      current = opened[0] #grab first element
      opened.pop(0) #pop first element

      x = current.x
      y = current.y

      if self.idx(x) == goal[0] and self.idx(y) == goal[1]:
        print("found path to goal in ", total_closed, " expansions")
        path = maze_path()
        path.came_from = came_from
        path.closed = closed
        path.final = current

        return path

      next_state = self.expand(current)

      for i in range(len(next_state)):
        g2 = next_state[i].g
        x2 = next_state[i].x
        y2 = next_state[i].y
        theta2 = next_state[i].theta

        if (x2 < 0 or x2 >= len(grid)) or (y2 < 0 or y2 >= len(grid[0])):
          # invalid cell
          continue

        stack2 = self.theta_to_stack_number(theta2)

        if closed[stack2][self.idx(x2)][self.idx(y2)] == 0 and grid[self.idx(x2)][self.idx(y2)] == 0:
          opened.append(next_state[i])
          closed[stack2][self.idx(x2)][self.idx(y2)] = 1
          came_from[stack2][self.idx(x2)][self.idx(y2)] = current
          total_closed += 1

    print("no valid path.")
    path = maze_path()
    path.came_from = came_from
    path.closed = closed
    path.final = state
    return path
      

In [4]:
def main():
    print("Finding path through grid:")
  
    # Creates an Empty Maze and for testing the number of expansions with it
    for i in range(len(GRID)):
        print(GRID[i][0], end='')
        for j in range(1, len(GRID[0])):
            print(",", GRID[i][j], end='')
        print()
  
    hbf = HBF()

    get_path = hbf.search(GRID, START, GOAL)

    show_path = hbf.reconstruct_path(get_path.came_from, START, get_path.final)

    print("show path from start to finish")
    for i in range(len(show_path) - 1, -1, -1):
        step = show_path[i]
        print(f"##### step {step.g} #####")
        print(f"x {step.x}")
        print(f"y {step.y}")
        print(f"theta {step.theta}")

In [5]:
main()

Finding path through grid:
0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0
0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0
0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0
0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0
0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0
0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1
0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1
0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1
0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1
0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1
0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
found path to goal in  12606  expansions
show path from start to finish
##### step 1 #####
x 1.45
y 0.0
theta 0.0
##### step 2 #####
x 2.9
y 0.0
theta 0.0
##### step 3 #####
x 4.35
y 0.0
theta 0.0
##### step 4 #####
x 5.8

# Implement Hybrid A* (solution)

Here is one possible implementation for Hybrid A* using the "distance to goal" heuristic function. In this implementation, we have added an **`f`** value to the `maze_s` struct, which is set in the `expand` function. Additionally, we've added two new functions: `heuristic` and `compare_maze_s`. The `compare_maze_s function` is used for comparison of `maze_s` objects (`f`) when sorting the `opened` stack.

To get an even lower number of expansions, try reducing `NUM_THETA_CELLS` in `hybrid_breadth_first.h` to reduce the total number of cells that can be searched in the `closed` array. Be careful though! Making `NUM_THETA_CELLS` too small might result in the algorithm being unable to find a path through the maze.

Another possibility for improvement is to use the regular A* algorithm to assign a cost value to each grid cell. This grid of costs can then be used as the heuristic function, which will lead to an extremely efficient search. If you are looking for additional practice, give this a try!

In [6]:
class maze_s:
  def __init__(self, g:int=0, f:int=0, x:float=0.0, y:float=0.0, theta:float=0.0):
    self.g = g
    self.f = f
    self.x = x
    self.y = y
    self.theta = theta

class HBF:
  def __init__(self):
    self.NUM_THETA_CELLS = 90
    self.SPEED = 1.45
    self.LENGTH = 0.5

  def compare_maze_s(self, item:maze_s) -> bool:
    return item.f
  
  def heuristic(self, x:float, y:float, goal:List[int]) -> float:
    return abs(y - goal[0]) + abs(x - goal[1]); #return grid distance to goal

  def theta_to_stack_number(self, theta: float) -> int:
    # Takes an angle (in radians) and returns which "stack" in the 3D 
    # configuration space this angle corresponds to. Angles near 0 go in the 
    # lower stacks while angles near 2 * pi go in the higher stacks.
    new_theta = (theta + 2 * math.pi) % (2 * math.pi)
    stack_number = int(round(new_theta * self.NUM_THETA_CELLS / (2 * math.pi))) % self.NUM_THETA_CELLS
    return stack_number

  def idx(self, float_num: float) -> int:
    # Returns the index into the grid for continuous position. So if x is 3.621, 
    # then this would return 3 to indicate that 3.621 corresponds to array 
    # index 3.
    return int(math.floor(float_num))

  def expand(self, state, goal: List[int]) -> List[maze_s]:
    g = state.g
    x = state.x
    y = state.y
    theta = state.theta

    g2 = g + 1
    next_states = []
    for delta_i in range(-35, 40, 5):
      delta = math.pi / 180.0 * delta_i
      omega = self.SPEED / self.LENGTH * math.tan(delta)
      theta2 = theta + omega
      if theta2 < 0:
        theta2 += 2 * math.pi
      x2 = x + self.SPEED * math.cos(theta)
      y2 = y + self.SPEED * math.sin(theta)
      f2 = g2 + self.heuristic(x2, y2, goal)
      state2 = maze_s(g2, f2, x2, y2, theta2) 
      next_states.append(state2)
    return next_states

  def reconstruct_path(self, came_from: List[List[List[maze_s]]], start: List[float], final: maze_s) -> List[maze_s]:
      path = [final]
      stack = self.theta_to_stack_number(final.theta)
      current = came_from[stack][self.idx(final.x)][self.idx(final.y)]
      stack = self.theta_to_stack_number(current.theta)

      x = current.x
      y = current.y

      while x != start[0] or y != start[1]:
        path.append(current)
        current = came_from[stack][self.idx(x)][self.idx(y)]
        x = current.x
        y = current.y
        stack = self.theta_to_stack_number(current.theta)
      return path

  def search(self, grid: List[List[int]], start: List[float], goal: List[int]):
    # Working Implementation of breadth first search. Does NOT use a heuristic
    # and as a result this is pretty inefficient. Try modifying this algorithm
    # into hybrid A* by adding heuristics appropriately.

    # TODO: Add heuristics and convert this function into hybrid A*
    closed = [[[0 for i in range(len(grid[0]))] for j in range(len(grid))] for k in range(self.NUM_THETA_CELLS)]
    came_from = [[[maze_s() for i in range(len(grid[0]))] for j in range(len(grid))] for k in range(self.NUM_THETA_CELLS)]
    theta = start[2]
    stack = self.theta_to_stack_number(theta)
    g = 0

    state = maze_s()
    state.g = g
    state.x = start[0]
    state.y = start[1]
    state.f = g + self.heuristic(state.x, state.y, goal)
    state.theta = theta
    
    closed[stack][self.idx(state.x)][self.idx(state.y)] = 1
    came_from[stack][self.idx(state.x)][self.idx(state.y)] = state
    total_closed = 1
    opened = [state]
    finished = False
    while opened:
      opened.sort(key=self.compare_maze_s)
      current = opened[0] #grab first element
      opened.pop(0) #pop first element

      x = current.x
      y = current.y

      if self.idx(x) == goal[0] and self.idx(y) == goal[1]:
        print("found path to goal in ", total_closed, " expansions")
        path = maze_path()
        path.came_from = came_from
        path.closed = closed
        path.final = current

        return path

      next_state = self.expand(current,goal)

      for i in range(len(next_state)):
        g2 = next_state[i].g
        x2 = next_state[i].x
        y2 = next_state[i].y
        theta2 = next_state[i].theta

        if (x2 < 0 or x2 >= len(grid)) or (y2 < 0 or y2 >= len(grid[0])):
          # invalid cell
          continue

        stack2 = self.theta_to_stack_number(theta2)

        if closed[stack2][self.idx(x2)][self.idx(y2)] == 0 and grid[self.idx(x2)][self.idx(y2)] == 0:
          opened.append(next_state[i])
          closed[stack2][self.idx(x2)][self.idx(y2)] = 1
          came_from[stack2][self.idx(x2)][self.idx(y2)] = current
          total_closed += 1

    print("no valid path.")
    path = maze_path()
    path.came_from = came_from
    path.closed = closed
    path.final = state
    return path
      

In [7]:
main()

Finding path through grid:
0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0
0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0
0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0
0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0
0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0
0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0
0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1
0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1
0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1
0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1
0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1
0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
found path to goal in  7715  expansions
show path from start to finish
##### step 1 #####
x 1.45
y 0.0
theta 0.0
##### step 2 #####
x 2.9
y 0.0
theta 0.0
##### step 3 #####
x 4.35
y 0.0
theta 0.0
##### step 4 #####
x 5.8
